# 150. PyTorch 다중 분류 손실 함수

## Categorical Crossentropy
    
<img src="https://gombru.github.io/assets/cross_entropy_loss/softmax_CE_pipeline.png" width=500 />

##  `nn.CrossEntropyLoss`

- `nn.LogSoftmax()` 와 `nn.NLLLoss()` 를 단일 class 로 combine한 것.

    즉, `nn.CrossEntropyLoss()` = `nn.NLLLoss(nn.LogSoftmax())`

### nn.CrossEntropyLoss

$$loss(x, class) = -\log(\frac{exp(x[class])}{\sum_{j}{exp(x[j])}}) = -x[class] + log(\sum_{j}exp(x[j]))$$

### nn.LogSoftmax

$$LogSoftmax(x_i) = \log(\frac{exp(x_i)}{\sum_{j}{exp(x_j)}})$$

### nn.NLLLoss (Negative Log Likelihood Loss)
$$l(x, y) = - \frac{1}{N} \sum_{1}^{N}l_n$$

### 결론적으로, Pytorch 에서는 Cross Entropy Loss 값의 결과를 얻기 위해 2가지 방식이 존재

In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
def NLLLoss(logs, targets):
    # targets와 동일한 크기의 텐서를 생성하고, 데이터 타입은 float으로 설정
    out = torch.zeros_like(targets, dtype=torch.float)
    # targets의 길이만큼 반복하여 각 샘플에 대해 처리
    for i in range(len(targets)):
        # logs[i][targets[i]]를 통해, i번째 샘플의 타겟 클래스에 해당하는 로그 확률을 선택합니다.
        out[i] = logs[i][targets[i]]
    # out 텐서의 원소들의 합을 취한 후, 음수를 취하고 샘플의 수로 나누어 평균을 구합니다.
    # 이는 Negative Log Likelihood Loss를 계산하는 과정입니다.
    return -out.sum() / len(out)

In [4]:
# class label 이 1 인 다중분류
# 이 텐서는 모델의 출력으로 볼 수 있으며, 각 요소는 특정 클래스에 속할 확률 또는 점수를 나타냅니다.
# 여기서는 10개의 다른 클래스에 대한 점수를 포함하고 있습니다.
x = torch.Tensor([[0.8982, 0.805, 0.6393, 0.9983, 0.5731, 0.0469, 0.556, 0.1476, 0.8404, 0.5544]])

# LongTensor는 정수형 텐서를 나타내며, 여기서는 클래스 레이블이 '1'임을 나타냅니다.
y = torch.LongTensor([1])

## 방법 1
### nn.CrossEntropyLoss 함수 사용

-  `NNLLoss(Log(Softmax))`

$-x[class] + log(\sum_{j}exp(x[j]))$

In [5]:
# torch.nn.CrossEntropyLoss를 인스턴스화합니다.
# 이 손실 함수는 소프트맥스와 NLLLoss(Negative Log Likelihood Loss)를 결합한 것입니다.
cross_entropy_loss = torch.nn.CrossEntropyLoss()

# 손실 함수를 사용하여 예측값 x와 실제 레이블 y 사이의 크로스 엔트로피 손실을 계산합니다.
# CrossEntropyLoss는 내부적으로 x에 대해 소프트맥스 함수를 적용하여 확률 분포를 얻고,
# 이 확률 분포와 실제 레이블 y 사이의 NLLLoss를 계산합니다.
loss = cross_entropy_loss(x, y)

# 계산된 손실값 출력
print(loss)

tensor(2.1438)


## 방법 2
### nn.LogSoftmax + NLLLoss

- LogSoftmax : Log + Softmax - 두 함수를 따로 적용한 것 보다 수학적 안정성이 좋다.

In [6]:
# torch.nn.LogSoftmax를 인스턴스화합니다.
# LogSoftmax는 입력 텐서에 대해 소프트맥스 함수를 적용한 후, 그 결과의 로그 값을 반환
# 'dim=1' 매개변수는 소프트맥스 함수가 적용될 차원
# 여기서는 각 샘플의 클래스 점수에 대해 소프트맥스 적용
log_softmax = torch.nn.LogSoftmax(dim=1)

# log_softmax를 사용하여 x에 대한 로그 소프트맥스 값을 계산
# 이 결과는 각 클래스에 대한 로그 확률을 나타냅니다.
x_log = log_softmax(x)

# 앞서 정의한 NLLLoss 함수를 사용하여,
# 로그 소프트맥스 값(x_log)과 실제 레이블(y) 사이의 음의 로그 가능도 손실(NLL Loss)을 계산합니다.
# NLLLoss는 모델의 예측 로그 확률과 실제 레이블 간의 차이를 측정합니다.
# 여기서는 모델의 예측 로그 확률(x_log)이 더 정확할수록, 즉 실제 레이블에 해당하는 로그 확률 값이 높을수록 손실이 줄어듭니다.
loss = NLLLoss(x_log, y)

# 계산된 손실값 출력
print(loss)

tensor(2.1438)


In [8]:
# torch.nn.NLLLoss()를 인스턴스화하고, 계산된 로그 소프트맥스 값(x_log)과
# 실제 레이블(y)을 사용하여 음의 로그 가능도 손실(Negative Log Likelihood Loss)을 계산합니다.
loss = nn.NLLLoss()(x_log, y)
print(loss)

tensor(2.1438)


### 사용상의 주의 사항

#### nn.CrossEntropyLoss 를 사용할 경우
-  nn.CrossEntropyLoss 내에 softmax 함수가 포함되어 있으므로 Neural Network의 마지막 activation 함수로 Softmax를 지정 않고, logit 만 출력한다.  

#### nn.NLLLoss 를 사용할 경우
- nn.NLLLoss 는 입력으로 확률 분포가 와야 하므로, Neural Network의 마지막 actiovation 함수로 LogSoftmax를 지정 한다.

In [10]:
# CrossEntropyLoss 인스턴스를 생성합니다. 이 손실 함수는 소프트맥스 함수를 적용한 뒤,
# 크로스 엔트로피 손실을 계산합니다.
cross_entropyloss = nn.CrossEntropyLoss()

# LogSoftmax 인스턴스를 생성합니다. 이 함수는 입력 텐서에 대해 로그 소프트맥스 함수를 적용합니다.
log_softmax = nn.LogSoftmax(dim=1)

# NLLLoss(Negative Log Likelihood Loss) 인스턴스를 생성합니다.
# 이 손실 함수는 로그 확률에 대한 NLL 손실을 계산합니다.
negative_LLLoss = nn.NLLLoss()

In [11]:
# 실제 레이블을 나타내는 텐서를 생성합니다.
Y = torch.tensor([0, 2, 1])

# 좋은 예측값을 나타내는 텐서를 생성합니다.
# 이 텐서는 각 샘플에 대해 모델이 예측한 클래스 점수를 담고 있습니다.
y_pred_good = torch.tensor([[2.0, 1.0, 0.1], [0.5, 1.0, 3.1], [0.3, 1.0, 0.1]])

# 나쁜 예측값을 나타내는 텐서를 생성합니다.
y_pred_bad  = torch.tensor([[0.1, 1.0, 2.5], [3.1, 1.0, 0.5], [3.1, 0.1, 2.5]])

print("argmax index")
# torch.max 함수를 사용하여 y_pred_good의 각 샘플에 대해 가장 높은 점수를 가진 클래스의 인덱스를 얻습니다.
_, pred1 = torch.max(y_pred_good, 1)
print(_, pred1)

# torch.max 함수를 사용하여 y_pred_bad의 각 샘플에 대해 가장 높은 점수를 가진 클래스의 인덱스를 얻습니다.
_, pred2 = torch.max(y_pred_bad, 1)
print(_, pred2)

argmax index
tensor([2.0000, 3.1000, 1.0000]) tensor([0, 2, 1])
tensor([2.5000, 3.1000, 3.1000]) tensor([2, 0, 0])


In [12]:
# CrossEntropy Loss
# nn.CrossEntropyLoss()는 내부적으로 로그 소프트맥스와 NLLLoss를 결합한 것으로,
# 좋은 예측과 나쁜 예측에 대한 크로스 엔트로피 손실을 계산합니다.
loss_good = cross_entropyloss(y_pred_good, Y)
loss_bad = cross_entropyloss(y_pred_bad, Y)
print('nn.CrossEntropyLoss() 사용 결과 :')
print(loss_good.item(), loss_bad.item())
print()

# NLLLoss
# 먼저 nn.LogSoftmax()를 사용하여 로그 소프트맥스 값을 계산한 후,
# nn.NLLLoss()로 네거티브 로그 우도 손실(NLLLoss)을 계산합니다.
m1 = log_softmax(y_pred_good)
m2 = log_softmax(y_pred_bad)

loss_good = negative_LLLoss(m1, Y)
loss_bad = negative_LLLoss(m2, Y)
print('nn.LogSoftmax() + nn.NLLLoss() 사용 결과 :')
print(loss_good.item(), loss_bad.item())

nn.CrossEntropyLoss() 사용 결과 :
0.41337862610816956 2.973893404006958

nn.LogSoftmax() + nn.NLLLoss() 사용 결과 :
0.41337862610816956 2.973893404006958


### forward method 의 return value

In [13]:
class Ex_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(2, 8)
        self.linear2 = nn.Linear(8, 3)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward1(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        # CrossEntropyLoss를 사용할 경우
        # no log softmax at the end
        return x

    def forward2(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        # NLLLoss 를 사용할 경우
        # log softmax 를 output 에 사용
        x = self.log_softmax(x)
        return x

In [14]:
model = Ex_NN()

x = torch.FloatTensor([[20, 10], [10, 30], [10, 1]])
y = torch.tensor([0, 2, 1])
x

tensor([[20., 10.],
        [10., 30.],
        [10.,  1.]])

- CrossEntropyLoss 사용

In [15]:
loss = nn.CrossEntropyLoss()
logits = model.forward1(x)
print(loss(logits, y))

tensor(8.5752, grad_fn=<NllLossBackward0>)


- Negative Log Likelihood Loss 사용

In [16]:
loss = nn.NLLLoss()
prob = model.forward2(x)
print(loss(prob, y))

tensor(8.5752, grad_fn=<NllLossBackward0>)


### category 분류

In [17]:
print(prob)

tensor([[-9.8463e+00, -5.7576e-05, -1.2279e+01],
        [-1.2482e+01, -3.9339e-06, -1.5843e+01],
        [-3.7094e+00, -3.6579e-02, -4.4719e+00]],
       grad_fn=<LogSoftmaxBackward0>)


In [22]:
# prob는 모델의 출력 확률 분포를 나타내는 텐서입니다.

# torch.argmax 함수는 주어진 차원(axis)에 대해 최대값을 갖는 인덱스를 반환합니다.
# 여기서 axis=-1은 텐서의 마지막 차원을 의미합니다. 다중 클래스 분류 문제에서는
# 각 샘플의 예측된 클래스 인덱스를 찾는 데 사용됩니다.
torch.argmax(prob, axis=-1)

tensor([1, 1, 1])

In [24]:
# `torch.max` 함수는 주어진 텐서에서 최대 값을 찾습니다.
# 이 함수는 두 개의 반환값을 가집니다: 최대 값과 해당 값의 인덱스입니다.
# `prob` 변수는 특정 확률 분포 또는 점수가 포함된 텐서를 가정합니다.
# `axis=-1` 매개변수는 최대 값을 찾을 차원을 지정합니다.
# `-1`은 텐서의 마지막 차원을 의미합니다. 이는 다차원 텐서에서 각 벡터별로 최대 값을 찾는 데 사용됩니다.
torch.max(prob, axis=-1)

torch.return_types.max(
values=tensor([-5.7576e-05, -3.9339e-06, -3.6579e-02], grad_fn=<MaxBackward0>),
indices=tensor([1, 1, 1]))